In [2]:
import pandas as pd
import zipfile
import io
from pathlib import Path

In [3]:
zip_path = "archive (3) 3.zip"

In [4]:
import os

In [5]:
zip_name = "archive (3) 3.zip" 
dfs = []
with zipfile.ZipFile(zip_name, "r") as zf:
    for name in zf.namelist():
        if name.endswith(".csv"):
            df = pd.read_csv(zf.open(name), encoding="latin1", on_bad_lines="skip")
            dfs.append(df)

merged = pd.concat(dfs, ignore_index=True)

In [6]:
import re

def to_float(s):
    s = str(s)
    s = re.sub(r'[^\d.\-]', '', s)  # keep only digits, dot, minus
    return float(s) if s else float('nan')

In [7]:
merged['category'].unique()
merged.rename(columns={'title': 'Brand'},inplace=True)


In [8]:
category_groups = {
    #  Apparel & Clothing
    "Apparel": [
        "bedsheets", "innerwear-vests", "bodysuit", "boxers", "bra", "briefs",
        "camisoles", "clothing-set", "co-ords", "dresses", "dungarees",
        "dupatta", "ethnic-dresses", "jackets", "jeans", "jumpsuit", "kurtas",
        "kurta-sets", "kurtis", "lounge-pants", "lounge-shorts", "lounge-tshirts",
        "nightdress", "night-suits", "palazzos", "sarees", "saree-blouse",
        "shirts", "shorts", "skirts", "sweaters", "sweatshirts", "swim-tops",
        "thermal-bottoms", "tights", "tops", "track-pants", "trousers",
        "trunk", "tshirts", "tunics"
    ],

    #  Footwear
    "Footwear": [
        "boots", "casual-shoes", "formal-shoes", "flats", "flip-flops",
        "heels", "sandals", "sports-shoes"
    ],

    #  Accessories & Jewelry
    "Accessories": [
        "belts", "bracelet", "caps", "earrings", "gloves", "hair-accessory",
        "mangalsutra", "ring", "scarves", "sunglasses", "wallets", "watches",
        "clutches", "handbags", "backpacks", "duffel-bag", "trolley-bag"
    ],

    #  Home & Living
    "Home & Living": [
        "bath-robe", "bathroom-accessories", "bath-towels",
        "bedsheets", "candle-holders", "chair-cover", "curtains-and-sheers",
        "cutlery", "dinnerware", "floor-mats--dhurries", "home-fragrance-set",
        "pillow-covers", "runners", "showpieces", "table-covers",
        "towel-set", "wall-art"
    ],

    #  Beauty & Personal Care
    "Beauty & Grooming": [
        "face-moisturisers", "face-wash-and-cleanser", "foundation",
        "hair-brush-and-comb", "hair-cream-and-mask", "hair-masks",
        "lipstick", "perfume-and-body-mist", "shampoo-and-conditioner",
        "shaving-essentials"
    ],

    #  Tech & Lifestyle
    "Tech & Lifestyle": [
        "headphones", "mobile-accessories", "outdoor-masks", "yoga-mats"
    ],

    #  Ethnic & Traditional Wear
    "Ethnic Wear": ["sherwani"],

    #  Miscellaneous
    "Other": []  # fallback
}


In [9]:
# Reverse the mapping to a lookup dict
category_lookup = {
    cat: group for group, cats in category_groups.items() for cat in cats
}

# Apply mapping to your dataframe
merged["category_group"] = merged["category"].map(category_lookup).fillna("Other")

In [10]:
merged["Initial_Price"] = merged["initial_price"] * 0.012
merged["Discount"] = merged["discount"] * 0.012
merged["Final_Price"] = merged["final_price"].apply(to_float)

In [11]:
print(merged.shape)
merged.head()

(2000, 29)


,product_id,Brand,product_description,rating,ratings_count,initial_price,discount,final_price,currency,images,...,seller_information,variations,best_offer,more_offers,category,Unnamed: 0,category_group,Initial_Price,Discount,Final_Price
0,18602872.0,My Room,Yellow & Green Floral 140 TC King Bedsheet wit...,4.7,10.0,2999.0,54.0,"""â¹2,999.00""",INR,http://assets.myntassets.com/assets/images/186...,...,NaN,[{}],{},"[{""offer_name"":""10% Instant Discount on Citi C...",bedsheets,NaN,Home & Living,35.988,0.648,2999.0
1,18602850.0,My Room,Ethnic Print 144 TC Super King Bedsheet with 2...,4.5,22.0,2999.0,54.0,"""â¹2,999.00""",INR,http://assets.myntassets.com/assets/images/186...,...,NaN,[{}],{},"[{""offer_name"":""10% Instant Discount on Citi C...",bedsheets,NaN,Home & Living,35.988,0.648,2999.0
2,8961147.0,Story@home,Pink & White Floral 160 TC Cotton 1 King Bedsh...,3.7,7.0,2199.0,53.0,"""â¹2,199.00""",INR,http://assets.myntassets.com/assets/images/896...,...,NaN,"[{},{}]",{},"[{""offer_name"":""10% Instant Discount on Citi C...",bedsheets,NaN,Home & Living,26.388,0.636,2199.0
3,8430275.0,MYTRIDENT,Sea Green Floral Flat 120 TC Cotton 1 Queen Be...,4.4,26.0,2199.0,46.0,"""â¹2,199.00""",INR,http://assets.myntassets.com/assets/images/pro...,...,NaN,"[{},{},{},{},{},{},{}]",{},"[{""offer_name"":""10% Instant Discount on Citi C...",bedsheets,NaN,Home & Living,26.388,0.552,2199.0
4,18346118.0,Arrabi,Pink & White Graphic 300 TC King Bedsheet with...,0.0,0.0,3329.0,74.0,"""â¹865.00""",INR,http://assets.myntassets.com/assets/images/183...,...,Oldâââ. 2âââ162âââââââ...,"[{},{},{},{},{}]","{""applicable_on"":""Orders above Rs. 999 (only o...","[{""offer_name"":""10% Instant Discount on IndusI...",bedsheets,NaN,Home & Living,39.948,0.888,865.0


In [12]:
## Must convert to float before adjusting currency

In [13]:
import re

def parse_money(s):
    # Make sure value is a string
    s = str(s)
    
    # Fix mojibake (â¹ -> ₹)
    try:
        s = s.encode("latin1").decode("utf-8")
    except:
        pass
    
    # Remove commas
    s = s.replace(",", "")
    
    # Strip everything except digits, minus, and dot
    s = re.sub(r"[^\d.-]", "", s)
    
    # Convert to float if possible
    return float(s) if s not in ("", ".", "-", "nan") else None

merged["Final_Price"] = merged["Final_Price"].map(parse_money)
print(merged[["Final_Price"]].head())

   Final_Price
0       2999.0
1       2999.0
2       2199.0
3       2199.0
4        865.0


In [14]:
import ast

def to_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        val = ast.literal_eval(str(x))
        return val if isinstance(val, list) else []
    except Exception:
        return []

merged["sizes_parsed"] = merged["sizes"].apply(to_list)

In [15]:
merged["size_count"] = merged["sizes"].apply(lambda x: len(x) if isinstance(x, list) else 0) 

In [16]:
type(merged["sizes_parsed"].iloc[0])

list

In [17]:
merged["size_count"] = merged["sizes_parsed"].apply(len)

In [18]:
cols_to_parse = ["Final_Price","Initial_Price"]

merged[cols_to_parse] = merged[cols_to_parse].map(parse_money)

In [19]:
import re

def to_float(s):
    s = str(s)
    s = re.sub(r'[^\d.\-]', '', s)  # keep only digits, dot, minus
    return float(s) if s else float('nan')

In [20]:
for col in ["Initial_Price", "Final_Price"]:
    if col in merged.columns:
        merged[col] = merged[col].apply(to_float)

In [21]:
merged["Final_Price"] = merged["Final_Price"] * 0.012

In [22]:
print(merged[["Final_Price"]].head())

   Final_Price
0       35.988
1       35.988
2       26.388
3       26.388
4       10.380


In [23]:
merged["Discount"] = merged["Initial_Price"] - merged["Final_Price"]

In [24]:
merged["Discount_Perc"] = (merged["Initial_Price"] - merged["Final_Price"]) / merged["Initial_Price"]

In [25]:
max_sizes = merged.groupby("category")["size_count"].transform("max")

In [26]:
merged["missing_sizes"] = max_sizes - merged["size_count"]

In [27]:
import numpy as np
merged["size_completeness"] = np.where(max_sizes.gt(0),
                                        merged["size_count"] / max_sizes,
                                        np.nan)

In [28]:
import ast
import pandas as pd
import numpy as np

# 1) Normalize images column to a Python list of URLs
def to_list_of_urls(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    s = str(x).strip()
    # Try JSON-ish first
    if s.startswith("["):
        try:
            v = ast.literal_eval(s)
            if isinstance(v, list):
                return [str(u) for u in v]
        except Exception:
            pass
    # Fallback: comma-separated string
    return [u.strip() for u in s.split(",") if u.strip()]

merged["images_list"] = merged["images"].apply(to_list_of_urls)

# 2) Clean up obvious junk (optional)
merged["images_list"] = merged["images_list"].apply(
    lambda L: [u for u in L if u.startswith("http")]  # keep valid-looking URLs
)

# 3) Count per product
merged["image_count"] = merged["images_list"].apply(len)

# 4) Category max and gaps
cat_max_imgs = merged.groupby("category")["image_count"].transform("max")
merged["missing_images"] = (cat_max_imgs - merged["image_count"]).astype("Int64")
merged["image_completeness"] = np.where(cat_max_imgs.gt(0),
                                        merged["image_count"] / cat_max_imgs,
                                        np.nan)
cat_max_sizes = merged.groupby("category")["size_count"].transform("max")
merged["size_completeness"] = np.where(
    cat_max_sizes.gt(0),
    merged["size_count"] / cat_max_sizes,
    np.nan
)


In [29]:
import re
import unicodedata
import pandas as pd

try:
    from ftfy import fix_text
    from unidecode import unidecode
    HAS_FTFY = True
except Exception:
    HAS_FTFY = False

def strip_nonprintable(s: str) -> str:
    return "".join(ch for ch in s if not unicodedata.category(ch).startswith("C"))

def normalize_seller(raw):
    if pd.isna(raw):
        return ""
    s = str(raw)

    # 1️⃣ Try to repair mojibake (the √¢¬ñ¬à garbage)
    if HAS_FTFY:
        s = fix_text(s)
    else:
        try:
            s = s.encode("latin-1", "ignore").decode("utf-8", "ignore")
        except Exception:
            pass
        s = re.sub(r"(√¢¬ñ¬à)+", "", s)

    # 2️⃣ Remove leftover non-printables & normalize unicode
    s = strip_nonprintable(s)
    s = unicodedata.normalize("NFKC", s)

    # 3️⃣ Clean up punctuation and spaces
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"[^\w&' -]", "", s)   # keep letters, digits, &, ', space, hyphen

    # 4️⃣ ASCII fallback
    if HAS_FTFY:
        s = unidecode(s)

    # 5️⃣ Final tidy
    s = s.title()
    return s or "Unknown Seller"

# Apply the cleaning
merged["seller_name"] = merged["seller_name"].apply(normalize_seller)

# Collapse very short/blank results
merged.loc[merged["seller_name"].str.len() < 3, "seller_name"] = "Unknown Seller"

# Quick sanity check
print(merged["seller_name"].value_counts().head(10))

seller_name
Unknown Seller    602
Supom              80
Vis St             58
Flatarmme          54
Arv Lityllim       52
Wesry Dind_M       42
Trut Cerc          32
Trum Ril           30
Ind Nee Pmit       26
Aarlifylelim       26
Name: count, dtype: int64


In [30]:
merged["missing_sizes_flag"] = (merged["missing_sizes"] > 0).astype(int)
merged["missing_images_flag"] = (merged["missing_images"] > 0).astype(int)
merged["discount_flag"] = (merged["Discount_Perc"] > 0).astype(int)


In [31]:
merged["rating"] = merged["rating"].astype(float)

In [32]:
merged["ratings_count"] = pd.to_numeric(merged["ratings_count"], errors="coerce").astype("Int64")

In [33]:
merged["value_score"] = merged["rating"] / merged["Final_Price"]

In [34]:
## Too long didnt read objective: Instead of dividing every single category by the same 0-5 star 
## rating use a multiplier based ont he average price of each category relative to the smallest category to make the value score more fair 


In [35]:
cat_mean_price   = merged.groupby("category")["Final_Price"].transform("mean")
baseline_mean    = cat_mean_price.groupby(merged["category"]).transform("min") 

In [36]:
multiplier = cat_mean_price / baseline_mean
price_idx  = merged["Final_Price"] / multiplier   # price normalized to baseline category
merged["value_score_catnorm"] = merged["rating"] / price_idx

In [37]:
merged["value_score"] = merged["value_score"] * 100 


In [38]:
cat_avg_price = merged.groupby("category")["Final_Price"].transform("mean")

# 2) Baseline as a SCALAR (cheapest category’s average) — not stored as a column
baseline_avg = float(cat_avg_price.min())

# 3) Multiplier per row (how much pricier this category is than the cheapest category)
multiplier = cat_avg_price / baseline_avg

# 4) Adjusted rating (ratings scaled by category price level)
merged["adjusted_rating"] = merged["rating"] * multiplier

# 5) Value score (adjusted rating per dollar), scaled ×100 for readability
merged["value_score"] = (merged["adjusted_rating"] / merged["Final_Price"]) * 100


print(merged.shape)
merged.head()

(2000, 44)


,product_id,Brand,product_description,rating,ratings_count,initial_price,discount,final_price,currency,images,...,images_list,image_count,missing_images,image_completeness,missing_sizes_flag,missing_images_flag,discount_flag,value_score,value_score_catnorm,adjusted_rating
0,18602872.0,My Room,Yellow & Green Floral 140 TC King Bedsheet wit...,4.7,10,2999.0,54.0,"""â¹2,999.00""",INR,http://assets.myntassets.com/assets/images/186...,...,[http://assets.myntassets.com/assets/images/18...,4,3,0.571429,0,1,0,145.778725,0.130599,52.462848
1,18602850.0,My Room,Ethnic Print 144 TC Super King Bedsheet with 2...,4.5,22,2999.0,54.0,"""â¹2,999.00""",INR,http://assets.myntassets.com/assets/images/186...,...,[http://assets.myntassets.com/assets/images/18...,4,3,0.571429,1,1,0,139.575375,0.125042,50.230386
2,8961147.0,Story@home,Pink & White Floral 160 TC Cotton 1 King Bedsh...,3.7,7,2199.0,53.0,"""â¹2,199.00""",INR,http://assets.myntassets.com/assets/images/896...,...,[http://assets.myntassets.com/assets/images/89...,4,3,0.571429,1,1,0,156.512580,0.140215,41.300540
3,8430275.0,MYTRIDENT,Sea Green Floral Flat 120 TC Cotton 1 Queen Be...,4.4,26,2199.0,46.0,"""â¹2,199.00""",INR,http://assets.myntassets.com/assets/images/pro...,...,[http://assets.myntassets.com/assets/images/pr...,3,4,0.428571,1,1,0,186.123068,0.166742,49.114155
4,18346118.0,Arrabi,Pink & White Graphic 300 TC King Bedsheet with...,0.0,0,3329.0,74.0,"""â¹865.00""",INR,http://assets.myntassets.com/assets/images/183...,...,[http://assets.myntassets.com/assets/images/18...,4,3,0.571429,1,1,1,0.000000,0.000000,0.000000


In [39]:
merged["price_tier"] = pd.qcut(
    merged["Final_Price"], 
    q=3, 
    labels=["Low", "Mid", "Premium"]
)

In [40]:
merged["value_tier"] = pd.qcut(
    merged["value_score"], 
    q=3, 
    labels=["Low", "Mid", "Premium"]
)

In [41]:
merged["Discount_Bin"] = pd.cut(
    merged["Discount_Perc"],
    bins=[0, 10, 25, 50, 75, 100],
    labels=["0–10%", "10–25%", "25–50%", "50–75%", "75–100%"]
)

In [42]:
merged['image_comp_tier'] = pd.cut(
    merged['image_completeness'],
    bins=[-0.001, 0.5, 0.85, 1.0],
    labels=['Low (≤50%)', 'Mid (50–85%)', 'High (≥85%)'])

In [43]:
merged["size_comp_tier"] = pd.cut(
    merged["size_completeness"],
    bins=[-0.001, 0.33, 0.66, 1.0],
    labels=["Low (≤33%)", "Mid (33–66%)", "High (≥66%)"]
)

In [47]:
import numpy as np
import pandas as pd

merged["ratings_count"] = pd.to_numeric(merged["ratings_count"], errors="coerce").fillna(0)

cat_med_nonzero = merged.groupby("category_group")["ratings_count"] \
                        .transform(lambda s: s[s > 0].median())

# If a group has *all* zeros, fall back to the global non-zero median
global_med_nonzero = merged.loc[merged["ratings_count"] > 0, "ratings_count"].median()
den = cat_med_nonzero.fillna(global_med_nonzero)

merged["RVR_category"] = 100 * merged["ratings_count"] / den

In [49]:
import pandas as pd

# Round ratings to nearest whole number (1–5)
merged["star_bucket"] = pd.cut(
    merged["rating"].round(), 
    bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
    labels=["1★", "2★", "3★", "4★", "5★"]
)

In [50]:
merged.groupby("category")["product_id"].nunique()

category
backpacks                4
bath-robe                1
bath-towels              1
bathroom-accessories     1
bedsheets               11
                        ..
tunics                   4
wall-art                 1
wallets                  7
watches                 15
yoga-mats                1
Name: product_id, Length: 97, dtype: int64

In [51]:
merged["star_bucket"].unique()

['5★', '4★', NaN, '3★', '2★']
Categories (5, object): ['1★' < '2★' < '3★' < '4★' < '5★']

In [52]:
merged["Discount_Perc"]    = merged["Discount_Perc"].round(2)

In [53]:
w = merged["ratings_count"].fillna(0)

# Overall weighted average rating
weighted_avg_rating = (merged["rating"] * w).sum() / np.maximum(w.sum(), 1)

# Add CSI as a constant column so it's available in Tableau
merged["CSI"] = 100 * (weighted_avg_rating - 1) / 4

In [54]:
import numpy as np

# weighted average rating per category_group
weighted_sum = merged["rating"] * merged["ratings_count"]
cat_num = weighted_sum.groupby(merged["category_group"]).transform("sum")
cat_den = merged["ratings_count"].groupby(merged["category_group"]).transform("sum").replace(0, np.nan)
merged["cat_weighted_avg_rating"] = cat_num / cat_den
# Weighted values per row
merged["weighted_sum_brand"] = merged["rating"] * merged["ratings_count"]

In [56]:


# CSI per category_group (0–100)
merged["CSI_category"] = 100 * (merged["cat_weighted_avg_rating"] - 1) / 4

In [57]:
avg_ratings_count = merged["ratings_count"].mean()
global_median_ratings_count = merged["ratings_count"].median()
merged["RVR"] = (avg_ratings_count / global_median_ratings_count) * 100
merged["has_discount"] = np.where(merged["discount_flag"] == 0, "No Discount", "Discounted")

In [ ]:
## group_medians = merged.groupby("category_group")["ratings_count"].transform("median")

In [58]:
cat_med_nonzero = merged.groupby("category_group")["ratings_count"] \
                        .transform(lambda s: s[s > 0].median()) ## the right one to use due to median being 0 for ethnic wear 
merged["RVR_category"] = (merged["ratings_count"] / cat_med_nonzero) * 100

In [61]:
cols_keep = [
    "product_id", "Brand", "category",
    "Final_Price", "Discount_Perc","Initial_Price","CSI_category",
    "rating", "ratings_count","category_group",
    "value_score", "adjusted_rating","has_discount","RVR_category",
    "size_count", "image_count","CSI","star_bucket","Total_Category_Savings","Avg_Product_Savings",
    "missing_sizes_flag", "missing_images_flag","RVR","Total_Weighted_Savings",
    "price_tier","value_tier","Discount_Bin","size_comp_tier","image_completeness",'image_comp_tier',"size_completeness",
]

merged_cleanoff = merged[cols_keep].copy()


In [62]:
merged_cleanoff.to_csv("TableauloadFinalOff1.csv", index=False)

In [63]:
print(merged_clean.shape)
merged_clean.head()

(2000, 31)


,product_id,Brand,category,Final_Price,Discount_Perc,Initial_Price,CSI_category,rating,ratings_count,category_group,...,missing_images_flag,RVR,Total_Weighted_Savings,price_tier,value_tier,Discount_Bin,size_comp_tier,image_completeness,image_comp_tier,size_completeness
0,18602872.0,My Room,bedsheets,35.988,0.00,35.988,81.625,4.7,10,Home & Living,...,1,451.123529,1861978.2,Premium,Mid,NaN,High (≥66%),0.571429,Mid (50–85%),1.000000
1,18602850.0,My Room,bedsheets,35.988,0.00,35.988,81.625,4.5,22,Home & Living,...,1,451.123529,1861978.2,Premium,Mid,NaN,Mid (33–66%),0.571429,Mid (50–85%),0.333333
2,8961147.0,Story@home,bedsheets,26.388,0.00,26.388,81.625,3.7,7,Home & Living,...,1,451.123529,1861978.2,Premium,Mid,NaN,Mid (33–66%),0.571429,Mid (50–85%),0.333333
3,8430275.0,MYTRIDENT,bedsheets,26.388,0.00,26.388,81.625,4.4,26,Home & Living,...,1,451.123529,1861978.2,Premium,Mid,NaN,Mid (33–66%),0.428571,Low (≤50%),0.333333
4,18346118.0,Arrabi,bedsheets,10.380,0.74,39.948,81.625,0.0,0,Home & Living,...,1,451.123529,1861978.2,Mid,Low,0–10%,Mid (33–66%),0.571429,Mid (50–85%),0.333333
